In [22]:
# import packages
import warnings
import pandas as pd
import numpy as np
import sklearn
import pickle
from sklearn import pipeline
from sklearn import model_selection
from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import ensemble
from sklearn import neural_network
from sklearn import decomposition

In [2]:
# toggle warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('default')

In [3]:
basic_features = pd.read_csv("../data/features.csv")
eng_features = pd.read_csv("../data/engineered_features.csv")
nn_features = pd.read_csv("../data/nn_features.csv")

In [ ]:
# raw features (wip)
# raw_features = pd.read_csv("../data/raw_features.csv")

In [5]:
# add mood to neural network dataframe and raw features dataframe
combined_nn = pd.concat([basic_features, nn_features], axis = 1)
# combined_raw = pd.concat([basic_features, raw_features], axis = 1)

combined_nn['song_check'] = combined_nn['title'] + ' - ' + combined_nn['artist']
# combined_raw['song_check'] = combined_raw['title'] + ' - ' + combined_raw['artist']

if combined_nn['song'].equals(combined_nn['song_check']):
    print ("Neural network features match.")
    combined_nn = combined_nn.drop(columns = ['song', 'song_check'])
else:
    print ("Neural network features DO NOT match.")
    
# if combined_raw['song'].equals(combined_raw['song_check']):
#     print ("Raw features match.")
#     combined_raw = combined_raw.drop(columns = ['song', 'song_check'])
# else:
#     print ("Raw features DO NOT match.")

Neural network features match.


In [6]:
basic_features

,title,artist,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid,primary,secondary
0,a new adventure,takanashi yasuharu,117.453835,-0.010635,0.082014,3.164051,0.186062,1,6
1,a oh,super junior,129.199219,-0.005782,0.078039,3.213386,0.179459,3,2
2,a thousand years,christina perri,92.285156,-0.016321,0.044617,3.217903,0.122203,6,5
3,adore u,seventeen,103.359375,-0.007745,0.058125,3.142738,0.145660,3,2
4,after rain,aimer,117.453835,-0.010324,0.068620,3.211794,0.147982,2,5
...,...,...,...,...,...,...,...,...,...
195,you he bu ke,xu song,99.384014,0.015339,0.056268,3.166332,0.156298,2,4
196,youth,troye sivan,123.046875,-0.002242,0.056377,3.192966,0.140693,5,6
197,zhe shi ai,"henry, donghae",151.999081,-0.020014,0.019520,3.202357,0.075014,4,6
198,zui mei hun li,bai xiao bai,112.347147,0.022650,0.051512,3.168251,0.134493,2,4


In [7]:
eng_features

,title,artist,tempo,chroma_number,zcr_mean,zcr_std,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta energy_std,delta mfcc_1_std,delta mfcc_3_std,delta mfcc_13_std,delta spectral_centroid_std,delta spectral_entropy_std,delta spectral_flux_std,delta spectral_spread_std,primary,secondary
0,a new adventure,takanashi yasuharu,117.453835,-0.010635,0.082014,0.031156,0.061664,3.164051,0.186062,0.201923,...,0.031983,0.619485,0.254529,0.189750,0.039689,0.304516,0.003464,0.026444,1,6
1,a oh,super junior,129.199219,-0.005782,0.078039,0.028662,0.087948,3.213386,0.179459,0.196234,...,0.031600,0.785050,0.345785,0.225282,0.035337,0.333457,0.002662,0.021355,3,2
2,a thousand years,christina perri,92.285156,-0.016321,0.044617,0.018092,0.052477,3.217903,0.122203,0.170703,...,0.016750,0.521221,0.241593,0.196299,0.028887,0.155287,0.002794,0.030025,6,5
3,adore u,seventeen,103.359375,-0.007745,0.058125,0.027802,0.113696,3.142738,0.145660,0.170424,...,0.064444,1.280268,0.469696,0.248865,0.035615,0.324714,0.007247,0.028047,3,2
4,after rain,aimer,117.453835,-0.010324,0.068620,0.026402,0.083934,3.211794,0.147982,0.169066,...,0.064776,1.005204,0.362243,0.220264,0.033298,0.300292,0.005283,0.025979,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,you he bu ke,xu song,99.384014,0.015339,0.056268,0.042850,0.073693,3.166332,0.156298,0.198088,...,0.041610,1.132257,0.402372,0.242564,0.071073,0.410712,0.009546,0.050956,2,4
196,youth,troye sivan,123.046875,-0.002242,0.056377,0.035249,0.081590,3.192966,0.140693,0.169824,...,0.036208,0.903736,0.388618,0.235086,0.039178,0.372678,0.005693,0.027476,5,6
197,zhe shi ai,"henry, donghae",151.999081,-0.020014,0.019520,0.004853,0.010187,3.202357,0.075014,0.148539,...,0.005241,1.118669,0.309914,0.191399,0.036687,0.037481,0.012192,0.058416,4,6
198,zui mei hun li,bai xiao bai,112.347147,0.022650,0.051512,0.039506,0.056785,3.168251,0.134493,0.169568,...,0.025834,1.111881,0.477303,0.251263,0.042643,0.410848,0.009373,0.031184,2,4


In [8]:
combined_nn

,title,artist,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid,primary,secondary,zcr_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,a new adventure,takanashi yasuharu,117.453835,-0.010635,0.082014,3.164051,0.186062,1,6,0.082014,...,0.005514,0.014898,0.006219,0.053885,0.003913,0.002354,0.006004,0.031340,0.001888,0.020060
1,a oh,super junior,129.199219,-0.005782,0.078039,3.213386,0.179459,3,2,0.078039,...,0.011865,0.009256,0.007407,0.061640,0.005371,0.013171,0.014533,0.026054,0.003867,0.016081
2,a thousand years,christina perri,92.285156,-0.016321,0.044617,3.217903,0.122203,6,5,0.044617,...,0.017381,0.014182,0.009949,0.022762,0.001656,0.005285,0.012485,0.016931,0.002702,0.007498
3,adore u,seventeen,103.359375,-0.007745,0.058125,3.142738,0.145660,3,2,0.058125,...,0.009794,0.016707,0.010291,0.073798,0.003834,0.004607,0.011580,0.044534,0.005407,0.018485
4,after rain,aimer,117.453835,-0.010324,0.068620,3.211794,0.147982,2,5,0.068620,...,0.012467,0.018193,0.011926,0.062329,0.002194,0.008356,0.026241,0.036722,0.002945,0.014673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,you he bu ke,xu song,99.384014,0.015339,0.056268,3.166332,0.156298,2,4,0.056268,...,0.021090,0.026517,0.025086,0.071825,0.002510,0.005989,0.014877,0.026483,0.003959,0.016674
196,youth,troye sivan,123.046875,-0.002242,0.056377,3.192966,0.140693,5,6,0.056377,...,0.013734,0.018895,0.010968,0.048100,0.004163,0.005751,0.013536,0.036632,0.008829,0.013790
197,zhe shi ai,"henry, donghae",151.999081,-0.020014,0.019520,3.202357,0.075014,4,6,0.019520,...,0.008392,0.014873,0.011848,0.021116,0.005053,0.003572,0.006446,0.010286,0.003860,0.007238
198,zui mei hun li,bai xiao bai,112.347147,0.022650,0.051512,3.168251,0.134493,2,4,0.051512,...,0.020704,0.020964,0.022000,0.064105,0.006130,0.006886,0.014403,0.036415,0.004233,0.017066


In [19]:
def build_xy(features, feature_type):
    if feature_type == "basic":
        x = features[['tempo', 'chroma_number', 'zero_crossing_rate', 'energy_entropy', 'spectral_centroid']]
    elif feature_type == "eng":
        # 2 to skip the title and artist, -2 to skip last two mood columns
        x = features.iloc[:, 2:(features.shape[1] - 2)]
    elif feature_type == "neural_net":
        # 9 is the index of zcr_mean, the first neural network feature
        x = features.iloc[:, 9:]
#     elif feature_type == "raw":
#         # 9 is the index of the first frame for raw features
#         x = features.iloc[:, 9:]
    else:
        print ("Invalid feature type.")
        return (None, None)
    
    y = features[['primary', 'secondary']]
    y['combined'] = y['primary'].astype(str) + y['secondary'].astype(str)
    return (x, y)

In [10]:
def evaluate_mood(y_true, y_predict):
    true_list = y_true.tolist()
    score_values = []
    for index, value in enumerate(y_predict):
        true_moods = true_list[index]
        if str(value) == true_moods[0]:
            score_values.append(1)
        elif str(value) in list(true_moods):
            score_values.append(0.5)
        else:
            score_values.append(0)
    return pd.Series(score_values)

In [ ]:
# code for using my custom mood scoring function
# predictions = model_selection.cross_val_predict(pipe, x, y['primary'], cv = 5)
# scores = evaluate_mood(y['combined'], predictions)
# print ("Accuracy of", round(np.mean(scores), 3), "on both primary and secondary moods")

In [35]:
# Support Vector Machine with basic features
x, y = build_xy(basic_features, "basic")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.39


In [20]:
# Support Vector Machine with engineered features
x, y = build_xy(eng_features, "eng")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.54


In [23]:
# Support Vector Machine with PCA transformed engineered features
x, y = build_xy(eng_features, "eng")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.525


In [24]:
# final basic SVM model
x, y = build_xy(basic_features, "basic")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***SVM Best Parameters***")
print (model.best_params_)

***SVM Best Parameters***
{'svc__C': 0.75, 'svc__decision_function_shape': 'ovr', 'svc__gamma': 5, 'svc__kernel': 'rbf'}


In [25]:
# final engineered SVM model
x, y = build_xy(eng_features, "eng")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***SVM Best Parameters***")
print (model.best_params_)

***SVM Best Parameters***
{'svc__C': 5, 'svc__decision_function_shape': 'ovr', 'svc__gamma': 0.1, 'svc__kernel': 'sigmoid'}


In [26]:
# final transformed and engineered SVM model
x, y = build_xy(eng_features, "eng")
svm_model = svm.SVC()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('svc', svm_model)])
param_grid = {
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape': ['ovr', 'ovo'],
    'svc__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50],
    'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***SVM Best Parameters***")
print (model.best_params_)

***SVM Best Parameters***
{'svc__C': 0.5, 'svc__decision_function_shape': 'ovr', 'svc__gamma': 1, 'svc__kernel': 'rbf'}


In [27]:
# Logistic Regression with basic features
x, y = build_xy(basic_features, "basic")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.44000000000000006


In [28]:
# Logistic Regression with engineered features
x, y = build_xy(eng_features, "eng")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.53


In [29]:
# Logistic Regression with PCA transformed engineered features
x, y = build_xy(eng_features, "eng")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.545


In [30]:
# final basic LR model
x, y = build_xy(basic_features, "basic")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***LR Best Parameters***")
print (model.best_params_)

***LR Best Parameters***
{'log_reg__C': 50, 'log_reg__penalty': 'l1', 'log_reg__solver': 'saga'}


In [31]:
# final engineered LR model
x, y = build_xy(eng_features, "eng")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***LR Best Parameters***")
print (model.best_params_)

***LR Best Parameters***
{'log_reg__C': 1, 'log_reg__penalty': 'l2', 'log_reg__solver': 'liblinear'}


In [32]:
# final transformed and engineered LR model
x, y = build_xy(eng_features, "eng")
lr_model = linear_model.LogisticRegression()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('log_reg', lr_model)])
param_grid = {
    'log_reg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'log_reg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'log_reg__C': [0.25, 0.5, 0.75, 1, 3, 5, 10, 25, 50]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***LR Best Parameters***")
print (model.best_params_)

***LR Best Parameters***
{'log_reg__C': 0.25, 'log_reg__penalty': 'l2', 'log_reg__solver': 'liblinear'}


In [33]:
# K Nearest Neighbor with basic features
x, y = build_xy(basic_features, "basic")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.4


In [34]:
# K Nearest Neighbor with engineered features
x, y = build_xy(eng_features, "eng")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.525


In [36]:
# K Nearest Neighbor with PCA transformed engineered features
x, y = build_xy(eng_features, "eng")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.55


In [37]:
# final basic KNN model
x, y = build_xy(basic_features, "basic")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***KNN Best Parameters***")
print (model.best_params_)

***KNN Best Parameters***
{'knn__metric': 'euclidean', 'knn__n_neighbors': 9, 'knn__weights': 'distance'}


In [38]:
# final engineered KNN model
x, y = build_xy(eng_features, "eng")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***KNN Best Parameters***")
print (model.best_params_)

***KNN Best Parameters***
{'knn__metric': 'manhattan', 'knn__n_neighbors': 9, 'knn__weights': 'distance'}


In [39]:
# final transformed and engineered KNN model
x, y = build_xy(eng_features, "eng")
knn_model = neighbors.KNeighborsClassifier()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('knn', knn_model)])
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis'],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***KNN Best Parameters***")
print (model.best_params_)

***KNN Best Parameters***
{'knn__metric': 'chebyshev', 'knn__n_neighbors': 9, 'knn__weights': 'distance'}


In [40]:
# Gaussian Naive Bayes with basic features
x, y = build_xy(basic_features, "basic")
nb_model = naive_bayes.GaussianNB()
param_grid = {
    'var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(nb_model, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.43


In [41]:
# Gaussian Naive Bayes with engineered features
x, y = build_xy(eng_features, "eng")
nb_model = naive_bayes.GaussianNB()
param_grid = {
    'var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(nb_model, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.5149999999999999


In [42]:
# Gaussian Naive Bayes with PCA transformed engineered features
x, y = build_xy(eng_features, "eng")
nb_model = naive_bayes.GaussianNB()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('gnb', nb_model)])
param_grid = {
    'gnb__var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.5199999999999999


In [43]:
# final basic GNB model
x, y = build_xy(basic_features, "basic")
nb_model = naive_bayes.GaussianNB()
param_grid = {
    'var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(nb_model, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***GNB Best Parameters***")
print (model.best_params_)

***GNB Best Parameters***
{'var_smoothing': 1e-07}


In [44]:
# final engineered GNB model
x, y = build_xy(eng_features, "eng")
nb_model = naive_bayes.GaussianNB()
param_grid = {
    'var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(nb_model, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***GNB Best Parameters***")
print (model.best_params_)

***GNB Best Parameters***
{'var_smoothing': 1e-06}


In [45]:
# final transformed and engineered GNB model
x, y = build_xy(eng_features, "eng")
nb_model = naive_bayes.GaussianNB()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('gnb', nb_model)])
param_grid = {
    'gnb__var_smoothing': [0.000000001, 0.00000001, 0.0000001, 0.000001],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***GNB Best Parameters***")
print (model.best_params_)

***GNB Best Parameters***
{'gnb__var_smoothing': 1e-09}


In [46]:
# Random Forest with basic features
x, y = build_xy(basic_features, "basic")
rf_model = ensemble.RandomForestClassifier()
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(rf_model, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.36000000000000004


In [47]:
# Random Forest with engineered features
x, y = build_xy(eng_features, "eng")
rf_model = ensemble.RandomForestClassifier()
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(rf_model, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.535


In [48]:
# Random Forest with PCA transformed engineered features
x, y = build_xy(eng_features, "eng")
rf_model = ensemble.RandomForestClassifier()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('forest', rf_model)])
param_grid = {
    'forest__max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'forest__min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'forest__max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.475


In [49]:
# final basic RF model
x, y = build_xy(basic_features, "basic")
rf_model = ensemble.RandomForestClassifier()
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(rf_model, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***RF Best Parameters***")
print (model.best_params_)

***RF Best Parameters***
{'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 2}


In [50]:
# final engineered RF model
x, y = build_xy(eng_features, "eng")
rf_model = ensemble.RandomForestClassifier()
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(rf_model, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***RF Best Parameters***")
print (model.best_params_)

***RF Best Parameters***
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 7}


In [51]:
# final transformed and engineered RF model
x, y = build_xy(eng_features, "eng")
rf_model = ensemble.RandomForestClassifier()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('forest', rf_model)])
param_grid = {
    'forest__max_depth': [5, 10, 15, 20, 25, 50, 75, None],
    'forest__min_samples_leaf': [1, 2, 5, 7, 10, 12, 15],
    'forest__max_features': ["sqrt", "log2"]
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***RF Best Parameters***")
print (model.best_params_)

***RF Best Parameters***
{'forest__max_depth': 50, 'forest__max_features': 'sqrt', 'forest__min_samples_leaf': 1}


In [26]:
# Multi-Layer Perceptron with neural network features
x, y = build_xy(combined_nn, "neural_net")
mlp_model = neural_network.MLPClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('mlp', mlp_model)])
param_grid = {
    'mlp__hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (60,)],
    'mlp__activation': ['logistic', 'tanh', 'relu', 'identity'],
    'mlp__solver': ['lbfgs', 'sgd', 'adam'],
    'mlp__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.5650000000000001


In [27]:
# final MLP model with neural network features
x, y = build_xy(combined_nn, "neural_net")
mlp_model = neural_network.MLPClassifier()
scaler = preprocessing.MinMaxScaler()
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('mlp', mlp_model)])
param_grid = {
    'mlp__hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (60,)],
    'mlp__activation': ['logistic', 'tanh', 'relu', 'identity'],
    'mlp__solver': ['lbfgs', 'sgd', 'adam'],
    'mlp__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
model.fit(x, y['primary'])
print ("***MLP Best Parameters***")
print (model.best_params_)

***MLP Best Parameters***
{'mlp__activation': 'logistic', 'mlp__alpha': 0.1, 'mlp__hidden_layer_sizes': (60,), 'mlp__solver': 'adam'}


In [52]:
# Multi-Layer Perceptron with raw features
# x, y = build_xy(combined_raw, "raw")
# mlp_model = neural_network.MLPClassifier()
# scaler = preprocessing.MinMaxScaler()
# pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('mlp', mlp_model)])
# param_grid = {
#     'mlp__hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (60,)],
#     'mlp__activation': ['logistic', 'tanh', 'relu', 'identity'],
#     'mlp__solver': ['lbfgs', 'sgd', 'adam'],
#     'mlp__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1],
# }
# model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
# accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
# print("Average accuracy:", np.mean(accuracies))

In [53]:
# final MLP model with raw features
# x, y = build_xy(combined_raw, "raw")
# mlp_model = neural_network.MLPClassifier()
# scaler = preprocessing.MinMaxScaler()
# pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('mlp', mlp_model)])
# param_grid = {
#     'mlp__hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (60,)],
#     'mlp__activation': ['logistic', 'tanh', 'relu', 'identity'],
#     'mlp__solver': ['lbfgs', 'sgd', 'adam'],
#     'mlp__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1],
# }
# model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
# model.fit(x, y['primary'])
# print ("***MLP Best Parameters***")
# print (model.best_params_)

In [71]:
# stacking
x, y = build_xy(eng_features, "eng")

svc_pipe = pipeline.make_pipeline(preprocessing.MinMaxScaler(), 
                                  svm.SVC(C = 5, decision_function_shape = 'ovr', gamma = 0.1, kernel = 'sigmoid'))
lr_pipe = pipeline.make_pipeline(preprocessing.MinMaxScaler(),
                                 decomposition.PCA(n_components = 0.95, svd_solver = "full"),
                                 linear_model.LogisticRegression(C = 0.25, penalty = 'l2', solver = 'liblinear'))
knn_pipe = pipeline.make_pipeline(preprocessing.MinMaxScaler(),
                                  decomposition.PCA(n_components = 0.95, svd_solver = "full"),
                                  neighbors.KNeighborsClassifier(metric = 'chebyshev', n_neighbors = 9, weights = 'distance'))
gnb_pipe = pipeline.make_pipeline(preprocessing.MinMaxScaler(),
                                  decomposition.PCA(n_components = 0.95, svd_solver = "full"),
                                  naive_bayes.GaussianNB(var_smoothing = 1e-09))
rf_pipe = pipeline.make_pipeline(preprocessing.MinMaxScaler(),
                                 ensemble.RandomForestClassifier(max_depth = None, max_features = 'log2', min_samples_leaf = 7))

estimators = [('svc', svc_pipe),
              ('lr', lr_pipe),
              ('knn', knn_pipe),
              ('gnb', gnb_pipe),
              ('rf', rf_pipe)]

stacker = ensemble.StackingClassifier(estimators = estimators, 
                                      final_estimator = linear_model.LogisticRegression(),
                                      passthrough = True)
accuracies = model_selection.cross_val_score(stacker, x, y['primary'], cv = 5)
print("Average primary mood accuracy:", np.mean(accuracies))
predictions = model_selection.cross_val_predict(stacker, x, y['primary'], cv = 5)
scores = evaluate_mood(y['combined'], predictions)
print ("Accuracy on both primary and secondary moods:", round(np.mean(scores), 3))



Average primary mood accuracy: 0.58
Accuracy on both primary and secondary moods: 0.665


In [72]:
predictions

array([1, 3, 4, 3, 3, 1, 2, 2, 1, 4, 5, 3, 5, 5, 1, 3, 2, 5, 3, 2, 5, 3,
       3, 1, 1, 4, 1, 3, 3, 2, 1, 1, 4, 4, 4, 3, 4, 3, 3, 5, 2, 2, 1, 1,
       3, 1, 5, 3, 3, 5, 5, 1, 4, 5, 1, 4, 2, 1, 5, 5, 2, 3, 5, 2, 3, 2,
       5, 1, 3, 2, 3, 3, 2, 5, 3, 2, 3, 4, 3, 2, 3, 3, 2, 1, 5, 5, 4, 2,
       2, 2, 2, 4, 4, 3, 2, 1, 3, 3, 2, 4, 2, 3, 2, 4, 4, 5, 4, 3, 4, 4,
       5, 4, 2, 5, 1, 3, 5, 2, 5, 4, 5, 3, 3, 2, 1, 5, 3, 3, 4, 2, 5, 2,
       5, 3, 5, 5, 5, 3, 4, 2, 4, 4, 3, 2, 1, 5, 5, 5, 3, 3, 1, 3, 2, 4,
       1, 5, 2, 1, 3, 5, 4, 3, 1, 2, 3, 4, 3, 4, 3, 5, 3, 1, 1, 5, 3, 2,
       3, 5, 5, 1, 5, 3, 5, 5, 3, 3, 3, 3, 5, 2, 4, 5, 2, 5, 3, 5, 5, 4,
       5, 2])

In [ ]:
# use pickle to store the final model
stacked_model = None
filename = 'mood_model.sav'
pickle.dump(stacked_model, open(filename, 'wb'))